# Homework No.3

## By Athar Pasha and Umair Cheema 

**Problem**: _Classify the given dataset into four classes using various methods and find the most optimal classification algorithm._ 

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc
from math import log
%matplotlib inline

## Preprocessing ##

### Prepare Dataframes ###

In [39]:
#Read the training data in a dataframe
df = pd.read_csv('trainingdata.txt', sep='\t',header=None)
#Create pseudo column names
column_names = []
for i in range(1,len(df.columns)+1):
    column_names.append("X"+str(i))
#Set Pseudo column names:
df.columns = column_names
#Read the class labels and add to data frame as Y
df_Y = pd.read_csv('trainingTruth.txt',sep='\t',header=None)


,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X325,X326,X327,X328,X329,X330,X331,X332,X333,X334
0,0.6343,0.3623,NaN,0.4771,0.1597,0.2117,0.9309,0.3619,0.5584,0.1471,...,0.7431,0.3330,0.1783,-0.7218,1.0397,0.3064,-0.1804,0.5108,-0.7427,0.7402
1,-0.1128,0.2567,-0.3150,0.0312,0.4733,0.1741,0.1306,0.1011,0.5484,0.1618,...,0.6729,0.6554,-0.0108,-0.1236,-0.2452,-0.0694,0.1850,0.9860,0.6855,0.7555
2,-0.1147,-0.2147,1.0796,0.6069,0.3323,0.8456,0.3082,0.7404,0.1146,0.3531,...,-0.1409,-0.0531,0.0121,0.7682,0.5060,-0.3720,0.0644,0.2841,0.0834,0.1460
3,-0.1393,0.3778,0.6667,1.1136,0.6970,-1.0491,0.1121,0.8550,-0.4056,-0.4072,...,0.3824,1.0743,0.1053,0.4585,-0.3990,0.5170,-0.0985,0.7276,0.0813,-0.2179
4,-0.1739,-0.2137,0.4118,0.2800,0.1626,0.4143,-0.0570,0.6324,1.0733,-0.4641,...,0.2810,-0.0898,-0.2685,0.8918,-0.3160,0.4253,-0.3345,-0.0639,0.2184,0.2293
5,0.3024,1.4105,0.7253,0.0859,0.2292,1.2045,0.3980,0.2835,-0.3578,0.9702,...,0.4773,0.5400,0.5926,0.7246,0.1638,0.0497,-0.1140,0.3765,-0.1323,-0.3070
6,-0.0169,1.1642,0.9674,0.3168,-0.2157,0.7080,0.4421,0.1710,0.7279,-0.3016,...,0.2273,0.0369,0.2574,0.6628,0.7408,0.1348,0.7865,0.0320,0.2085,1.2017
7,0.0602,-0.0452,0.1252,0.2605,1.6316,0.6583,0.2071,-0.0453,0.9383,-0.1174,...,0.8475,0.1201,-0.2705,1.1758,0.5764,0.5759,0.1474,0.9546,0.2494,0.4189
8,-0.0128,0.6521,1.1530,-0.0040,0.7283,-0.0724,0.7828,0.5813,0.2787,-0.0740,...,1.5694,0.1903,0.9076,-0.2722,0.6499,-0.1173,0.7845,0.8120,0.5159,1.2287
9,0.5071,0.9207,-1.0091,-0.6333,0.1690,0.8713,0.6055,-0.6022,0.4773,0.3599,...,0.4492,0.3439,0.1825,0.3009,-0.2094,-0.4301,0.1052,0.1841,0.1583,0.6494


### Handle Missing Values ###

To start simple, we are just droping the rows containing N/A as a first step.

In [40]:
#Remove missing values
df_dropped_missing = df.dropna()


In [41]:
#What percentage of data is dropped
print ("Percentage Data dropped:",(1-(df_dropped_missing.shape[0]/df.shape[0]))*100,"%")

Percentage Data dropped: 66.86039820462653 %


Dropping 67% data is not a good idea as in real word data collection is often very expensive. This indicates that we should probably use Imputation to fill in the missing values.

In [43]:
#Create a dataframe with imputed values
imputer = Imputer(missing_values='NaN',strategy = 'median', axis=0)
df_imputed_missing = pd.DataFrame(imputer.fit_transform(df),columns = column_names)
#Add back the response variable
df_imputed_missing['Y'] = df_Y
df = df_imputed_missing
df

ValueError: Shape of passed values is (335, 17378), indices imply (334, 17378)

In [25]:
corrdf=df.corr().abs()
c=corrdf.values
u=corrdf.unstack()
corord = u.order(kind="quicksort")
print (corord[-4470:-4460])

X138  X79     0.044114
X22   X260    0.044130
X260  X22     0.044130
X149  X2      0.044135
X2    X149    0.044135
X30   X142    0.044135
X142  X30     0.044135
X248  X105    0.044158
X105  X248    0.044158
X163  X158    0.044170
dtype: float64


C:\Users\atharp\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: order is deprecated, use sort_values(...)


In [32]:
X=df.drop('Y',axis=1)
y=df['Y']
X=X.apply(np.log1p)
X

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X325,X326,X327,X328,X329,X330,X331,X332,X333,X334
0,0.491215,0.309174,0.357010,0.390081,0.148161,0.192024,0.657986,0.308881,0.443660,0.137237,...,0.555665,0.287432,0.164073,-1.279415,0.712803,0.267275,-0.198939,0.412639,-1.357513,0.554000
1,-0.119685,0.228489,-0.378336,0.030723,0.387505,0.160502,0.122748,0.096310,0.437222,0.149971,...,0.514559,0.504043,-0.010859,-0.131933,-0.281302,-0.071926,0.169743,0.686123,0.522062,0.562754
2,-0.121829,-0.241689,0.732176,0.474307,0.286907,0.612804,0.268652,0.554115,0.108496,0.302398,...,-0.151870,-0.054562,0.012027,0.569962,0.409457,-0.465215,0.062411,0.250058,0.080104,0.136278
3,-0.150009,0.320488,0.510846,0.748393,0.528862,NaN,0.106250,0.617885,-0.520203,-0.522898,...,0.323821,0.729624,0.100117,0.377409,-0.509160,0.416735,-0.103695,0.546733,0.078164,-0.245773
4,-0.191039,-0.240417,0.344865,0.246860,0.150659,0.346635,-0.058689,0.490051,0.729142,-0.623808,...,0.247641,-0.094091,-0.312658,0.637529,-0.379797,0.354382,-0.407217,-0.066033,0.197539,0.206445
5,0.264209,0.879834,0.545401,0.082409,0.206364,0.790501,0.335043,0.249591,-0.442855,0.678135,...,0.390216,0.431782,0.465368,0.544995,0.151691,0.048504,-0.121038,0.319544,-0.141909,-0.366725
6,-0.017044,0.772051,0.676713,0.275205,-0.242964,0.535323,0.366100,0.157858,0.546907,-0.358963,...,0.204817,0.036235,0.229046,0.508503,0.554345,0.126456,0.580258,0.031499,0.189380,0.789230
7,0.058458,-0.046253,0.117961,0.231508,0.967592,0.505793,0.188221,-0.046358,0.661811,-0.124883,...,0.613833,0.113418,-0.315396,0.777396,0.455144,0.454827,0.137499,0.670186,0.222663,0.349882
8,-0.012883,0.502047,0.766862,-0.004008,0.547138,-0.075155,0.578185,0.458247,0.245844,-0.076881,...,0.943672,0.174205,0.645846,-0.317729,0.500715,-0.124770,0.579138,0.594431,0.416009,0.801418
9,0.410187,0.652690,NaN,-1.003211,0.156149,0.626633,0.473435,-0.921806,0.390216,0.307411,...,0.371012,0.295576,0.167631,0.263056,-0.234963,-0.562294,0.100026,0.168983,0.146953,0.500412


In [28]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
from sklearn.tree import DecisionTreeClassifier
dtree=DecisionTreeClassifier()
dtree.fit(X_train,y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
predictions = dtree.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)


In [ ]:
rfc_pred = rfc.predict(X_test)
print(confusion_matrix(y_test,rfc_pred))

In [ ]:
print(classification_report(y_test,rfc_pred))